This ipynb shows an example (credited to Equinor) of using `get_1site` API to solve the "1-site-N-wells" problem, or the "site-level" layout optimization.  
"1-site-N-wells" problem means to find the best drill site location with optimized wellbore trajectories fulfilling various constraints,  
so that the total cost(length) of the N wellbores is minimum.  

In this example, there are 3 wells to be drilled from 1 site.  
Some optional input parameters are provided in this example, but commented. You can try different parameters to see the effect on the result.

# 0. Initiate environment

In [1]:
import requests
import json
import numpy as np
from plotly import graph_objects as go
import time

In [2]:
import sys
import os
rootpath = os.path.join(os.getcwd().split("WelLayout_API") [0], 
                        "WelLayout_API")
# print(rootpath)
sys.path.append(rootpath)

from API import get_1site
from tools.input2json import input2json
from tools.PlotSurvey_plotly import PlotSurvey_plotly as PlotSurvey
from tools.PlotContour_plotly import PlotContour_plotly as PlotContour, surface2mesh3d, PlotDangerArea

# 1. Prepare Data

## 1.1 Target information
The completion intervals (targets) are normally decided by the reservoir engineers based on the reservoir layer's trend to maximize the total production of the wells.  
Here we need the location of the first point of the completion interval($PT$) and its desired entry direction($VT$) for each well.

Following are the data for 3 targets. Multiple wells' information is gathered in a Matrix.

In [3]:
# 3 wells to be designed
n=3

# location of the first point of the completion interval (PT), [X, Y, Z]
PTM= np.array ([[410.9, 209.89, -3850.27],
                [3011.47, 2098.01, -4368.09],
                [1784.37, 763.8, -4179.39],])

# drilling direction at the first point of the completion interval (VT),
VTM= np.array ([[2.64, 1.48, -28.85],
                [-16.42, -10.47, -8.11],
                [4.83, 4.05, -28.0]])


## 1.2 KOP information
The KOP depth is normally determined by the drilling engineer based on the rock mechanical property of the formation to ensure the wellbore stabilty.  
Normally, the direction at KOP is vertically downwards.

In [4]:
# location of the KOP (PK), [X, Y, Z]
# set X, Y as np.nan, so that the API can find the best site location
# KOP is beneath site
PKM= np.array ([[np.nan, np.nan, -300.0],
                [np.nan, np.nan, -300.0],
                [np.nan, np.nan, -300.0]])

# drilling direction at the KOP (VK), [X, Y, Z]
VKM= np.array ([[0.0, 0.0, -1.0],
                [0.0, 0.0, -1.0],
                [0.0, 0.0, -1.0]])

## 1.3 DLS constraint
Dogleg severity (DLS) is constrained by the drilling equipment.  
Drilling engineers need to set a maximum allowable DLS to ensure the feasibility for drilling and the wellbore stability.  
DLS unit is given in `°/30m`

In the site-level case, we adopt the simplest wellbore trajectory where there are at most 2 curved sections, one at the buildup section after the KOP and the other at the landing section before the Target.

In [5]:
# DLS at the buildup section after the KOP 
dls_KOP=2.0

# DLS at the landing section before the Target
dls_Target=2.0

# format DLS for API
DLSM= np.array ([[dls_KOP, dls_Target],
                [dls_KOP, dls_Target],
                [dls_KOP, dls_Target]])

# optional: get the corresponding curvature radius of the DLS
rM= 30*180/DLSM/np.pi
print(rM)

[[859.4366927 859.4366927]
 [859.4366927 859.4366927]
 [859.4366927 859.4366927]]


## 1.4 [Optional] Other constraints
A constraint function is a function involving the parameters:  
`PK`, KOP location, 3D.  
`PT`, target location, 3D.  
`angDK`, turning angle(°) of the buildup section after the KOP, scalar.  
`angDT`, turning angle(°) of the landing section before the target, scalar.  
`P_L`, point on the trajectory at the given depth, 3D.  
`inclD_L`, inclination(°) of the trajectory at the given depth, scalar, [0, 180).  
`aziD_L`, azimuth(°) of the trajectory at the given depth, scalar, [0,360).  
...  
(**contact the author for more parameters related to the trajectory structure to build more complex engineering constraints**)  

Constraints for each well are gathered in a list.
All constraint functions are gathered in `neconM`.

$neconM \ge 0$

In this example, we do not have any other constraint.

In [6]:
neconM=None

### 1.4.1 constraints for site location
The site location constraint is a function of [X, Y] of the site 2D coordinate.  
We simplify all wells' KOP points are the same, and exactly beneath the site location, hence the site location can be presented by $PK$.  
X: `PK[0]`  
Y: `PK[1]`  

As for the 1-site-N-wells problem, all wells are drilled from the same site, hence the site location constraints for one well are also active for the others.
This means you just need to write the constraint functions for one well only.

In [7]:
# # uncomment the cell for site location constraints
# # The following constraints mean that the site location [X, Y] must fulfill X<516000, Y<6782000
# neconM=[
#         ["PK[0]+150", "-PK[0]+850", "PK[1]+100", "-PK[1]+1250",], # constraints for the 1st wellbore
#         None, # constraints for the 2nd wellbore
#         None # constraints for the 3rd wellbore
#         ]

### 1.4.2 constraints for trajectory tortuosity

The trajectory tortuosity constraint is a function of `angDK` and `angDT`.  
Limiting the turning angles in the trajectory is critical for controlling the torque & drag.

In [8]:
# # uncomment the cell for tortuosity constraints

# neconM=[
#         None, # constraints for the 1st wellbore
#         ["-angDT+90"], # constraints for the 2nd wellbore: turning angle of the landing section must be smaller than 90°
#         None # constraints for the 3rd wellbore
#         ]

### 1.4.3 constraints at certain depths (layers)

The layer constraint is a function of `P_L`, `inclD_L` and `aziD_L`.  
For each layer, the contraints are collected in a dictionary of two keys: `layer` and `con`.  
`layer` is a scalar depth value，or scattered 3D points (not recommended, as it's time consuming to deal discretized data in optimization).  
`con` is a list of constraint functions.

In [9]:
lay_conM=None

In [31]:
# uncomment the cell for layer constraints
# The following constraint means at the depth of 1500m, the inclination of the 2nd wellbore should not exceed 36°

lay_conM=[
        None, # constraints for the 1st wellbore
        [{'layer': -1500.0, 'con': ["-inclD_L+36"]}], # constraints for the 2nd wellbore
        None # constraints for the 3rd wellbore
        ]

In [32]:
# # uncomment the cell for layer constraints
# # The following constraint means at the depth of 1500m, there is a danger zone centered at [2450, 1550] with radius 100m

# lay_conM=[
#         [{'layer': -1500.0, 'con': ["(x_L-2450)**2+(y_L-1550)**2-100000"]}], # constraints for the 1st wellbore
#         [{'layer': -1500.0, 'con': ["(x_L-2450)**2+(y_L-1550)**2-100000"]}], # constraints for the 2nd wellbore
#         [{'layer': -1500.0, 'con': ["(x_L-2450)**2+(y_L-1550)**2-100000"]}] # constraints for the 3rd wellbore
#         ]

In [33]:
# # uncomment the cell for layer constraints
# # multiple constraints for the same layer

# lay_conM=[
#         [{'layer': -1500.0, 'con': ["(x_L-2450)**2+(y_L-1550)**2-100000"]}], # constraints for the 1st wellbore
#         [{'layer': -1500.0, 'con': ["-inclD_L+36", "(x_L-2450)**2+(y_L-1550)**2-100000"]}], # constraints for the 2nd wellbore
#         [{'layer': -1500.0, 'con': ["(x_L-2450)**2+(y_L-1550)**2-100000"]}], # constraints for the 3rd wellbore
#         ]

Plot and show the danger area

In [34]:
# x = np.arange(0, 3000, 20)
# y = np.arange(0, 3000, 20)
# X, Y = np.meshgrid(x, y)
# Z=np.full_like(X, lay_conM[1][0]['layer'], dtype=np.float64)
# for i in range(Z.shape[0]):
#     for j in range(Z.shape[1]):
#         x_L=X[i,j]
#         y_L=Y[i,j]
#         mask=eval(lay_conM[1][0]['con'][-1])
#         if mask>=0:
#             Z[i,j]=np.nan
            
# fig_danger=PlotDangerArea(X,Y,Z)

## 1.5 [Optional] Objective(cost) function
User-defined cost function for the trajectory, default is the length of the trajectory.


In [35]:
ObjM=None
# 
# ObjM=["L", "L", "L"] # same a default


You can also define any cost function for the trajectory.  
The function is related with `Ls`, `Lc` and `incl`.  
`Ls` is the length of the straight section,   
`Lc` is the length of the curved section,  
`incl` is the inclination angle (rad) of the straight section between the two curved section, rad.

In [36]:
# # uncomment the following lines to see different results
# ObjM=["2*Lc+Ls*(1+np.sin(incl))", # Lc: length of the curved section.
#       "2*Lc+Ls*(1+np.sin(incl))", # Ls: length of the straight section.
#       "2*Lc+Ls*(1+np.sin(incl))"] 

In [37]:
# # uncomment the following lines to see different results
# ObjM=["2*Lc+Ls", # Lc: length of the curved section.
#       "2*Lc+Ls", # Ls: length of the straight section.
#       "2*Lc+Ls", 
#       "2*Lc+Ls"] 

## 1.6 [Optional] Computational parameters 
Normally, you do not need to bother yourself about these paramters. API will automatically apply default values.

In [38]:
# discretized measured depth for trajectory data
MD_intervalM= None, # shape (n, ), default: 30m

In [39]:
# plot canvas
XRange= None, # shape (2, ), default: [min(X) of given points(PKM, PTM) - max(rM),  max(X) of given points(PKM, PTM) + max(rM)]
YRange= None, # shape (2, )

In [40]:
# grid resolution for cost contour
resolution= None, # scalar, default is 100. This affects the computational time significantly because of the cost contour's grid node values.

In [41]:
# radius of the cost contour for each well
cst_radiusM= None, # shape (n, ), default is 3000

# 2. Use the API to do the computation

## 2.1 save the input data to JSON

In [42]:
# save to current path, file name "input.json"
filepath="input.json"

input_data=input2json(n,                
        PTM,
        VTM,
        PKM,
        VKM,
        DLSM,
        ObjM=ObjM,
        neconM=neconM,
        lay_conM=lay_conM,

        MD_intervalM=MD_intervalM,
        XRange=XRange,
        YRange=YRange,
        resolution=resolution,
        cst_radiusM=cst_radiusM,
        
        filepath=filepath)

=====file input.json written successfully=====


## 2.2 prepare the parameters for calling API

If you have already saved the Json input before, load it directly.

In [43]:
# # load saved input data (json) for computation
# with open(filepath) as json_file:
#     input_data = json.load(json_file)

Define the output data file path for saving the response from the API server

In [44]:
output_filepath="output.json" # filepath for saving the output

## 2.3 fetch the response from the API

Without additional constraints, this case takes about 15 seconds to get the result.  
The result includes the 3 trajectories and 3 cost contours for the 3 wells, as well as 1 cost contour for the site.

In [45]:
tic=time.time()
output=get_1site(input_data, 
                filepath=output_filepath)
toc=time.time()
print(f"Elapsed time: {toc-tic} seconds")

requesting from https://home-test.make234.com/api/v1/get_1site
Status code: 200
Response content has been saved to "output.json"
Elapsed time: 10.479793071746826 seconds


# 3. Visualize the results

## 3.1 load and check the data

In [46]:
# # Load the output JSON file
# with open(output_filepath, "r") as file:
#     output = json.load(file)

In [47]:
# Extract the trajectory data 
wellbores = output["data"]["Trajectories"]

# Cost contour data
contour_1site=output['data']['CostASite']
for key in contour_1site.keys():
    contour_1site[key]=np.array(contour_1site[key], dtype=float)

In [48]:
# Loop through each wellbore
for wellbore in wellbores:
    # check the target location in the ouput
    print([wellbore['X'][-1], wellbore['Y'][-1], wellbore['Z'][-1]])

# check the target location in the input
# the data in input and output should match
input_data['FIELDOPT INPUT BLOCK']['PTM']

[410.9000000000001, 209.89000000000004, -3850.2700000000004]
[3011.4700000000003, 2098.0100000000007, -4368.09]
[1784.37, 763.8000000000001, -4179.39]


{'DESCRIPTION': 'target location, i.e., the 1st point of completion interval. 3D, [EAST,NORTH,Depth]',
 'UNIT': 'm',
 'VALUE': [[410.9, 209.89, -3850.27],
  [3011.47, 2098.01, -4368.09],
  [1784.37, 763.8, -4179.39]]}

## 3.2 3D Visualization
Plot the wellbore trajectories and the cost contour for the site.  
The cost contour indicates the total cost to reach all the targets while placing the drill site at the given [X, Y] location.  
If the cost contour at a given point is empty, then it means the site located at this point cannot reach at least one of the targets under the given constraints.

In [49]:
try:
    fig1=fig_danger
except:
    fig1=None

In [50]:
# Plot wellbore trajectory
# fig1=None #initiate a fig
styles=['k-', 'r-', 'g-', 'b-'] # line style for each trajectory
for i, wellbore in enumerate(wellbores):
    fig1=PlotSurvey(wellbore, # trajectory data
                    fig=fig1, # figure handle
                    style=styles[i], # line style for the trajectory
                    name=f'Well #{i+1}', # name of the trajectory in the plot
                    show=0) # wait for plottiing all trajectories

# plot the cost contour for the site
PlotContour(X=contour_1site['X'],
            Y=contour_1site['Y'],
            Contour_Val=contour_1site['cost'],
            fig=fig1,
            name='cost contour',
            azim=-65, # view angle azimuth
            elev=40, # view angle elevation
            show=0)

# find the minimum cost and its location (grid value)
mincost=np.nanmin(contour_1site['cost'])
try:
    minidx=np.nanargmin(contour_1site['cost'])
except:
    minidx=None
print (f"min cost (node value): {mincost}")
print (f"min cost site location(node location): {[contour_1site['X'][minidx], contour_1site['Y'][minidx]]}")

# show the figure
fig1.show()

min cost (node value): 12813.705500585907
min cost site location(node location): [1900.0, 1200.0]


save the interactive 3D figure

In [51]:
figpath="figure.html"
fig1.update_layout(
            autosize=True,  # auto resize the fig1 in webbrowers
            width=None,
            height=None,
            margin=dict(l=0, r=0, t=0, b=0),)
fig1.write_html(figpath, full_html=True, div_id="plotly-div", config={"responsive": True})
print(f"figure saved to file: \"{figpath}\"")

figure saved to file: "figure.html"
